# 1. Get miRNAs that are in (+) strand

In [16]:
# get all mRNA gene names from gencode.gtf
!grep -i protein_coding ../binfo1-datapack1/gencode.gtf | awk '$3=="gene" && $7=="+" {print $10, $1, $4, $5}' > mirna_dm.txt

miRNA 중 (+) strand인 entity 들만 선정

In [2]:
import pandas as pd
import re
from collections import Counter
from scipy.stats import entropy
from tqdm import tqdm
from datetime import datetime
import pickle

model_name = 'get_Lin28a_binding_seqs'
version = 2.8
logger_file = f'log/{model_name}_v{version}'


def logger(text, show=True):
    text = f'[{datetime.now().replace(microsecond=0)}] {text}'
    with open(f'{logger_file}.log', 'a') as f:
        f.write(text+'\n')
    if show:
        print(text)

In [4]:
mrnas = pd.read_csv('mirna_dm.txt', delimiter=' ', header=None)
mrnas.columns = ['gene_id','chr','start','end']
mrnas[mrnas['chr']!='chrM']
mrnas

,gene_id,chr,start,end
0,ENSMUSG00000025903.15;,chr1,4878011,4918633
1,ENSMUSG00000104217.2;,chr1,4878115,4956993
2,ENSMUSG00000033813.16;,chr1,4928037,4968132
3,ENSMUSG00000033793.13;,chr1,5140241,5232752
4,ENSMUSG00000025905.15;,chr1,5658689,5676354
...,...,...,...,...
10902,ENSMUSG00000064360.1;,chrM,9459,9806
10903,ENSMUSG00000065947.1;,chrM,9877,10173
10904,ENSMUSG00000064363.1;,chrM,10167,11544
10905,ENSMUSG00000064367.1;,chrM,11742,13565


In [5]:
mrnas['gene_id'] = mrnas['gene_id'].str.split('.',expand=True)
mrnas # all gene ids are unique

,gene_id,chr,start,end
0,ENSMUSG00000025903,chr1,4878011,4918633
1,ENSMUSG00000104217,chr1,4878115,4956993
2,ENSMUSG00000033813,chr1,4928037,4968132
3,ENSMUSG00000033793,chr1,5140241,5232752
4,ENSMUSG00000025905,chr1,5658689,5676354
...,...,...,...,...
10902,ENSMUSG00000064360,chrM,9459,9806
10903,ENSMUSG00000065947,chrM,9877,10173
10904,ENSMUSG00000064363,chrM,10167,11544
10905,ENSMUSG00000064367,chrM,11742,13565


In [6]:
def count_base(seq):
    counts = Counter(seq.upper())
    counts = [counts['A'], counts['T'], counts['G'], counts['C']]
    index = counts.index(max(counts))
    return counts, index

In [8]:
threshold_entropy = 1.2

genes = []
seqs = []


for i in tqdm(range(len(mrnas))):
    gene = mrnas.iloc[i]
    chr = gene['chr']
    start = gene['start']
    end = gene['end']
    gid = gene['gene_id']
    
    query = f'{chr}:{start}-{end}'
    file = f'pileups/gene_{i}.pileup'

    !samtools view -b -o - ../binfo1-datapack1/CLIP-35L33G.bam $query | samtools mpileup - > $file

  0%|          | 0/10907 [00:00<?, ?it/s]

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


  0%|          | 1/10907 [00:00<1:14:47,  2.43it/s]

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


  0%|          | 2/10907 [00:00<1:30:15,  2.01it/s]

...

100%|█████████▉| 10905/10907 [39:01<00:00,  4.29it/s]

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


100%|█████████▉| 10906/10907 [39:01<00:00,  4.03it/s]

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


100%|██████████| 10907/10907 [39:01<00:00,  4.66it/s]


In [ ]:
with open(f'seqs_mrna_{version}.pkl', 'wb') as f:
    pickle.dump((genes, seqs), f)
#   
# with open(f'seqs_mrna_{version}.pkl', 'rb') as f:
#     a,b = pickle.load(f)
    
# len(a)